#### Importing libraries

In [1]:
import numpy as np
import os
import cv2
import face_recognition as fr
from datetime import datetime

#### Importing images

In [2]:
path = r"C:\Users\Dell\ML projects\Image processing\FaceRecognitionProject\ImagesAttendance"

#list to hold all subject faces
images = [] #creating list of images we import
#list to hold labels for all subjects
classNames = [] #names of image file
myList = os.listdir(path)
print(myList)

['Aamir Khan.jpg', 'Salman Khan.jpg', 'Shah Rukh Khan.jpg', 'Shreyansh.jpg']


In [3]:
#we use these names to import images one by one
for cl in myList: #importing each class names & removing file extension
    curImg = cv2.imread(f'{path}/{cl}') 
    images.append(curImg) #appending images in empty image list
    classNames.append(os.path.splitext(cl)[0])
print(classNames)

['Aamir Khan', 'Salman Khan', 'Shah Rukh Khan', 'Shreyansh']


#### Image encoding process 

In [4]:
#defining a function to compute encodings
def findEncodings(images):
    encodeList = [] #creating empty list that will have all encoded images 
    for img in images:  #loop through all the images
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)  #convert it into RGB 
        #encoding face that we've detected
        encode = fr.face_encodings(img)[0]
        encodeList.append(encode)

    return encodeList

encodeListKnown = findEncodings(images) #encoded list for known faces 
print(len(encodeListKnown))
print("Encodings completed")

4
Encodings completed


#### Marking attendance

In [5]:
#defining function for marking attendance
def markAttendance(name):
    with open(r"C:\Users\Dell\ML projects\Image processing\FaceRecognitionProject/Attendance.csv",'r+') as f:
        myDataList = f.readlines() #reading all the lines we've currently in our data
        #print(myDataList)# 
        nameList = []
        for line in myDataList: #go through each line one by one
            entry = line.split(',') #splitting based on comma"," then we've name & time separated
            nameList.append(entry[0]) #appending entry of 1st element (i.e "Name") in empty list
        if name not in nameList: #if name is present then no changes would be made & if not then 
            now = datetime.now() #creating datetime object #gives date & time
            tString = now.strftime('%H:%M') #time format
            dString = now.strftime('%d-%m-%Y')
            f.writelines(f'\n{name}, {tString}, {dString}') 
#markAttendance("a")       

#### Image comparing

In [6]:
#find matches bw encodings but here we don't have image to match with.
#now that image will be coming from our webcam

#initialise the webcam
cap = cv2.VideoCapture(0)

#loop to get each frame one by one
while True:
    success,img = cap.read()
    imgS = cv2.resize(img,(0,0),None,0.25,0.25)
    imgS = cv2.cvtColor(imgS,cv2.COLOR_BGR2RGB) 
    
    facesLocCurframe = fr.face_locations(imgS) 
    encodeCurFrame = fr.face_encodings(imgS,facesLocCurframe)
    
    for encodeFace,faceLoc in zip(encodeCurFrame,facesLocCurframe):
        matches = fr.compare_faces(encodeListKnown,encodeFace)
        faceDist = fr.face_distance(encodeListKnown,encodeFace)
        print(faceDist)
        
        #lowest value in the index will be best match so
        matchIndex = np.argmin(faceDist)
        
        #displaying boundary box & name around face
        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            print(name)
            y1,x2,y2,x1 = faceLoc
            y1,x2,y2,x1 = y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,240,0),2)
            cv2.rectangle(img,(x1,y2-30),(x2,y2),(0,240,0),-1) #displaying filled box below the rectangle
            cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_SIMPLEX,(1/2),(255,255,255),2)
            markAttendance(name)
            
    cv2.imshow("Live Face",img)
    k = cv2.waitKey(100) & 0xff 
    if k == 27: # Press 'ESC' for exiting video
        break
    elif (k == ord('c')):
        # Save the captured image into the datasets folder
        cv2.imwrite(r"C:\Users\Dell\ML projects\Image processing\FaceRecognitionProject/User_" + str(name) + ".jpg",img)
        break

# Do a bit of cleanup
print("\n [INFO] Exiting Program ")
cap.release()
cv2.destroyAllWindows()

[0.48983171 0.649358   0.6947672  0.68622694]
AAMIR KHAN
[0.80100677 0.67016724 0.68146808 0.38096741]
SHREYANSH
[0.78433278 0.6750338  0.66773161 0.4332694 ]
SHREYANSH
[0.48977013 0.65440942 0.65598933 0.70223681]
AAMIR KHAN
[0.41337868 0.6389167  0.69983519 0.7345298 ]
AAMIR KHAN
[0.45544996 0.68127516 0.68282599 0.72224104]
AAMIR KHAN
[0.44132153 0.65450241 0.69253137 0.73492072]
AAMIR KHAN
[0.39923161 0.64490685 0.72266265 0.74125756]
AAMIR KHAN
[0.38748139 0.6416398  0.71722412 0.74003339]
AAMIR KHAN
[0.37882093 0.66063381 0.74001731 0.73023272]
AAMIR KHAN
[0.39901761 0.67410498 0.72841983 0.73889346]
AAMIR KHAN
[0.40757923 0.66895969 0.7344305  0.76284798]
AAMIR KHAN
[0.38741681 0.66863987 0.74607359 0.74611296]
AAMIR KHAN
[0.36145689 0.65946052 0.74931089 0.73434556]
AAMIR KHAN
[0.38050023 0.67591162 0.7492787  0.74465109]
AAMIR KHAN
[0.39118758 0.66365508 0.74597271 0.74847767]
AAMIR KHAN
[0.39011925 0.67463373 0.77508573 0.79234326]
AAMIR KHAN
[0.39705059 0.65317376 0.72155776